## Testing Geoapify Places API

In [ ]:
import sys
import pandas as pd
import numpy as np
# change working directory to the root of the project
sys.path.append('../')
from src.geoapify_api import geoapify_pipeline
from src.methods.data_ingestion import get_openweather, format_forecast_openweather
from src.helpers.utils import relative_distance, convert_easting_northing_to_long_lat
import importlib

#### Postcode input to be converted to longitude and latitude.

Postcode data was downloaded from: https://osdatahub.os.uk/downloads/open/CodePointOpen

In [ ]:
postcode_path = '../data/postcode_data/Data/'
# create function for getting postcode data
def get_postcode_data(input_postcode: str, postcode_path) -> pd.DataFrame:
    """Get postcode data from postcode csv files"""
    # Check if postcode starts with two or one letters
    if input_postcode[:2].isalpha() == True:
        # match first two letters of postcode to file
        postcode_file = postcode_path + input_postcode[:2] + '.csv'
    else:
        # match first letter of postcode to file
        postcode_file = postcode_path + input_postcode[0] + '.csv'
    # read in file
    postcode_df = pd.read_csv(postcode_file, names=['postcode', 'pqi', 'easting', 'northing'], header=None, usecols=[0, 1, 2, 3])
    #filter to input postcode
    postcode_df = postcode_df[postcode_df['postcode'] == input_postcode]
    # convert easting and northing to longitude and latitude
    postcode_df['latitude'], postcode_df['longitude'] = convert_easting_northing_to_long_lat(postcode_df['easting'].iloc[0], postcode_df['northing'].iloc[0])
    return postcode_df

# test function
postcode = 'E14 3BX'
post_df = get_postcode_data(postcode, postcode_path)
post_df

In [ ]:
importlib.reload(sys.modules['src.geoapify_api'])
lat = post_df['latitude'].iloc[0]
lon = post_df['longitude'].iloc[0]
request_params = {
    "apiKey": "493bbfea1f124b27906cf052aa330680",
    "categories": "tourism, leisure, entertainment",
    "limit": "500",
}
filters = {
        "circle": f"{lon},{lat},5000",
}

url = 'https://api.geoapify.com/v2/places?'

df, res = geoapify_pipeline(url, request_params, filters)

In [ ]:
# In category column filter to look for ['tourism,'theatre'] and remove ['artwork','memorial']
df1 = df[df['category_keywords'].apply(lambda x: 'tourism' in x or 'theatre' in x)]
df1 = df1[df1['category_keywords'].apply(lambda x: 'artwork' not in x and 'memorial' not in x)]
# remove rows containing either only 'sights' or only ['sights','tourism'] in either order
df1 = df1[~df1['category_keywords'].apply(lambda x: x == ['sights'] or x == ['sights','tourism'] or x == ['tourism','sights'])]
# apply relative distance function to dataframe
df1['distance_from_start'] = df1.apply(lambda x: relative_distance(lat, lon, x['lat'], x['lon']), axis=1)
# sort dataframe by distance
df1 = df1.sort_values(by='distance_from_start')
df1

In [ ]:
obs, forecast = get_openweather(lat=float(lat), lon=float(lon))
weather_df = format_forecast_openweather(forecast.forecast.to_dict())

In [ ]:
weather_df

In [ ]:
from src.helpers.time_tracker import timings_to_dataframe
timings_to_dataframe()